In [1]:
import requests
import camelot
import datetime
import os
import pandas as pd
from deta import Deta
from dotenv import load_dotenv
import constants

load_dotenv()

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))

data = deta.Drive("data")


In [2]:
url = 'https://www.dropbox.com/s/ckijmipu33z3feg/HourlyReport.pdf?dl=1'
r = requests.get(url, allow_redirects=True)
open('hourlyreport.pdf', 'wb').write(r.content)

1048650

In [4]:
# tables = camelot.read_pdf('hourlyreport.pdf', flavor='stream', pages='1', columns=['41, 57, 75, 97.5, 115.5, 138, 160, 187, 202, 220, 238, 256, 279, 301, 323, 342, 360, 381, 400, 425, 443, 465, 483, 501, 519, 537, 559.5, 577.5, 600, 627.5, 652, 667, 690, 706.5, 730, 753, 778, 798, 822, 839.5, 855.5, 881, 906, 930, 956, 978'])

In [3]:
tables = camelot.read_pdf('hourlyreport.pdf', pages= '1', flavor='stream', table_areas=['8,410,1000,50'])
tables[0].df

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,10/17/24,20,6,134,6,91,12,225,4,32,...,23,14,4,0,5,0,0,3,18,12
1,,19,7,128,3,85,10,213,1,28,...,18,14,4,0,6,0,0,0,17,11
2,,18,5,121,3,82,8,203,1,27,...,20,16,6,0,6,0,1,0,17,11
3,,17,12,116,3,79,15,195,4,26,...,19,18,4,0,7,0,5,5,16,11
4,,16,8,104,8,76,16,180,2,22,...,19,15,5,0,6,0,0,3,17,12
5,,15,8,96,7,68,15,164,0,20,...,16,11,6,0,7,0,0,2,18,11
6,,14,11,88,11,61,22,149,3,20,...,23,18,5,0,14,0,0,1,18,11
7,,13,16,77,11,50,27,127,3,17,...,20,15,2,0,16,0,0,3,20,13
8,,12,12,61,12,39,24,100,5,14,...,15,9,1,0,15,0,0,1,18,13
9,,11,10,49,8,27,18,76,1,9,...,9,3,3,0,14,0,0,1,17,13


In [ ]:
df = tables[0].df.reset_index(drop=True)
df.columns = constants.column_names
df.dateflg = df.dateflg[df.dateflg.str.strip() != '']
df.dateflg = df.dateflg.ffill()
for column in df.columns.tolist():
    if column in ['dateflg']:
        continue
    df[column] = df[column].astype('float').astype('int')
df["ds"] = pd.to_datetime(
    df["dateflg"] + " " + (df["timeflg"] - 1).astype(str) + ":00", format='mixed') + datetime.timedelta(hours=1)
df = df.set_index('ds').reset_index().drop(['dateflg','timeflg'], axis=1)


In [5]:
df.head()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
0,2024-10-17 20:00:00,6,134,6,91,12,225,4,32,0,...,23,14,4,0,5,0,0,3,18,12
1,2024-10-17 19:00:00,7,128,3,85,10,213,1,28,0,...,18,14,4,0,6,0,0,0,17,11
2,2024-10-17 18:00:00,5,121,3,82,8,203,1,27,0,...,20,16,6,0,6,0,1,0,17,11
3,2024-10-17 17:00:00,12,116,3,79,15,195,4,26,2,...,19,18,4,0,7,0,5,5,16,11
4,2024-10-17 16:00:00,8,104,8,76,16,180,2,22,1,...,19,15,5,0,6,0,0,3,17,12


In [ ]:
df.to_csv('current.csv', index=False)

In [9]:
data.put(name='current.csv', path='current.csv')

'current.csv'